<a href="https://colab.research.google.com/github/TBab0/duth_cv4/blob/main/1sratch%20shear_range.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import os
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import datetime

In [2]:
drive.mount('/content/drive')
local_zip = '/content/drive/MyDrive/imagedb_btsd.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close()

train_dir = '/content/imagedb'
test_dir = '/content/imagedb_test'

Mounted at /content/drive


In [3]:
ts = 32
# train_datagen = ImageDataGenerator(preprocessing_function=vgg16.preprocess_input, validation_split=0.2)
train_datagen  = ImageDataGenerator(rescale=1./255,
                                    shear_range=0.2,
                                    validation_split=0.2)

# --------------------
# Flow training images in batches of 20 using train_datagen generator
# --------------------
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=128,
                                                    class_mode='categorical',
                                                    color_mode='grayscale',
                                                    target_size=(ts,ts),
                                                    subset='training', seed=1)     
# --------------------
# Flow validation images in batches of 20 using test_datagen generator
# --------------------
validation_generator =  train_datagen.flow_from_directory(train_dir,
                                                        batch_size=128,
                                                        class_mode='categorical',
                                                         color_mode='grayscale',
                                                         target_size=(ts,ts),
                                                        subset='validation', seed=1)

Found 2457 images belonging to 34 classes.
Found 599 images belonging to 34 classes.


In [4]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(64, (5, 5), strides=(1, 1), activation='relu', input_shape=(ts, ts, 1)),
  tf.keras.layers.Conv2D(64, (5, 5), strides=(1, 1), activation='relu', input_shape=(ts, ts, 1)),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
  # tf.keras.layers.Conv2D(64, (5, 5), strides=(1, 1), activation='relu', input_shape=(ts, ts, 1)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(34, activation='softmax')
])
model.compile(optimizer = tf.keras.optimizers.Adam(), loss = 'categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 64)        1664      
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        102464    
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 64)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 9216)              0         
                                                                 
 dense (Dense)               (None, 512)               4719104   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                        

In [5]:
callbacks = []

save_best_callback = tf.keras.callbacks.ModelCheckpoint(f'best_weights.hdf5', save_best_only=True, verbose=1)
callbacks.append(save_best_callback)

early_stop_callback = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True, verbose=1)
callbacks.append(early_stop_callback)

history = model.fit(train_generator, validation_data=validation_generator, epochs=30, verbose=1, callbacks=callbacks)

Epoch 1/30
20/20 [==============================] - ETA: 0s - loss: 2.4148 - accuracy: 0.3940
Epoch 1: val_loss improved from inf to 1.12997, saving model to best_weights.hdf5
20/20 [==============================] - 10s 131ms/step - loss: 2.4148 - accuracy: 0.3940 - val_loss: 1.1300 - val_accuracy: 0.7713
Epoch 2/30
20/20 [==============================] - ETA: 0s - loss: 0.8224 - accuracy: 0.7977
Epoch 2: val_loss improved from 1.12997 to 0.39146, saving model to best_weights.hdf5
20/20 [==============================] - 2s 110ms/step - loss: 0.8224 - accuracy: 0.7977 - val_loss: 0.3915 - val_accuracy: 0.9098
Epoch 3/30
20/20 [==============================] - ETA: 0s - loss: 0.3342 - accuracy: 0.9133
Epoch 3: val_loss improved from 0.39146 to 0.31334, saving model to best_weights.hdf5
20/20 [==============================] - 2s 106ms/step - loss: 0.3342 - accuracy: 0.9133 - val_loss: 0.3133 - val_accuracy: 0.9349
Epoch 4/30
20/20 [==============================] - ETA: 0s - loss: 0.

In [6]:
test_datagen  = ImageDataGenerator(rescale=1./255)
test_generator =  test_datagen.flow_from_directory(test_dir,
                                                         batch_size=32,
                                                         class_mode  = 'categorical',
                                                         color_mode='grayscale',
                                                         target_size=(ts,ts)) 
loss, acc = model.evaluate(test_generator)
print(acc)


Found 2149 images belonging to 34 classes.
68/68 [==============================] - 1s 14ms/step - loss: 0.2528 - accuracy: 0.9558
0.9557933807373047
